## Notebook to map a new product by its name to Master Mapping excell and also to generate aligned paths (different regions and channels) table (in Py dataframe to be exported to excel) of Products Categories vs. IDs per Channel & Region for the input Product Name inserted in the first Column
Jul 2020 by Rafal Rakowski

In [1]:
# Create compute environment

import pandas as pd
from pandas import Series,DataFrame
from googletrans import Translator
import numpy as np
import re
from collections import OrderedDict

In [2]:
# Check path
%pwd

'/Users/Raf/ecommerce_practical/Cross_Mapping_Project'

In [3]:
# Read in with API the Master_Mapping.xlsx into Python Pandas
master_mapper = pd.ExcelFile('Master_Mapping.xlsx')
data_frame = master_mapper.parse('Sheet1')

In [4]:
# Create empty output dataframe for the new products mapping 
df_output = pd.DataFrame(columns=data_frame.columns) 
count = 0 
df_output.insert(0, 'Product Name', [])
df_output 

,Product Name,ID,Master Path,Amazon UK ID,Amazon UK Path,Amazon DE ID,Amazon DE Path,Amazon FR ID,Amazon FR Path,Amazon IT ID,...,Ebay ES Path,Ebay US,Ebay US Path,Ebay CA ID,Ebay CA Path,Ebay AU ID,Ebay AU Path,Test Channel,Test Path,Unnamed: 36


In [5]:
# Show a few lines of Master spreadsheet
data_frame

,ID,Master Path,Amazon UK ID,Amazon UK Path,Amazon DE ID,Amazon DE Path,Amazon FR ID,Amazon FR Path,Amazon IT ID,Amazon IT Path,...,Ebay ES Path,Ebay US,Ebay US Path,Ebay CA ID,Ebay CA Path,Ebay AU ID,Ebay AU Path,Test Channel,Test Path,Unnamed: 36
0,1,/Categories/3D Printing & Scanning/3D Printer ...,6.283024e+09,/Categories/3D Printing & Scanning/3D Printer ...,6.589298e+09,Elektronik & Foto/Computer & Zubehör/Computer-...,6.606606e+09,Informatique/Accessoires/Composants pour impri...,6.572429e+09,Informatica/Accessori/Ricambi e accessori per ...,...,Informática y tablets > Impresión 3D > Impreso...,260315.0,Computers/Tablets & Networking > 3D Printers &...,183066.0,Computers/Tablets & Networking > 3D Printers &...,183066,Computers/Tablets & Networking > 3D Printers &...,12345.0,This isnt working,NaN
1,2,/Categories/3D Printing & Scanning/3D Printer ...,6.283025e+09,/Categories/3D Printing & Scanning/3D Printer ...,6.589297e+09,Elektronik & Foto/Computer & Zubehör/Computer-...,6.606607e+09,Informatique/Accessoires/Composants pour impri...,6.572430e+09,Informatica/Accessori/Ricambi e accessori per ...,...,Informática y tablets > Impresión 3D > Impreso...,183066.0,Computers/Tablets & Networking > 3D Printers &...,183066.0,Computers/Tablets & Networking > 3D Printers &...,183066,Computers/Tablets & Networking > 3D Printers &...,NaN,NaN,NaN
2,3,/Categories/3D Printing & Scanning/3D Printer ...,6.283027e+09,/Categories/3D Printing & Scanning/3D Printer ...,6.589296e+09,Elektronik & Foto/Computer & Zubehör/Computer-...,6.606609e+09,Informatique/Accessoires/Composants pour impri...,6.572432e+09,Informatica/Accessori/Ricambi e accessori per ...,...,Informática y tablets > Impresión 3D > Impreso...,183066.0,Computers/Tablets & Networking > 3D Printers &...,183066.0,Computers/Tablets & Networking > 3D Printers &...,183066,Computers/Tablets & Networking > 3D Printers &...,NaN,NaN,NaN
3,4,/Categories/3D Printing & Scanning/3D Printer ...,8.624609e+09,/Categories/3D Printing & Scanning/3D Printer ...,8.623223e+09,Elektronik & Foto/Computer & Zubehör/Computer-...,8.641317e+09,Informatique/Accessoires/Composants pour impri...,8.641300e+09,Informatica/Accessori/Ricambi e accessori per ...,...,Informática y tablets > Impresión 3D > Impreso...,183066.0,Computers/Tablets & Networking > 3D Printers &...,183066.0,Computers/Tablets & Networking > 3D Printers &...,183066,Computers/Tablets & Networking > 3D Printers &...,NaN,NaN,NaN
4,5,/Categories/3D Printing & Scanning/3D Printer ...,6.283026e+09,/Categories/3D Printing & Scanning/3D Printer ...,6.589293e+09,Elektronik & Foto/Computer & Zubehör/Computer-...,6.606608e+09,Informatique/Accessoires/Composants pour impri...,6.572431e+09,Informatica/Accessori/Ricambi e accessori per ...,...,Informática y tablets > Impresión 3D > Impreso...,183066.0,Computers/Tablets & Networking > 3D Printers &...,183066.0,Computers/Tablets & Networking > 3D Printers &...,183066,Computers/Tablets & Networking > 3D Printers &...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48464,48465,Watches/Men/Wrist Watches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48465,48466,Watches/Women/Pocket Watches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48466,48467,Watches/Women/Watch Bands,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48467,48468,Watches/Women/Wrist Watches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Show Metadata of Master spreadsheet
data_frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48469 entries, 0 to 48468
Data columns (total 37 columns):
ID                 48469 non-null int64
Master Path        48469 non-null object
Amazon UK ID       22160 non-null float64
Amazon UK Path     22162 non-null object
Amazon DE ID       21443 non-null float64
Amazon DE Path     19368 non-null object
Amazon FR ID       21094 non-null float64
Amazon FR Path     18346 non-null object
Amazon IT ID       21097 non-null float64
Amazon IT Path     17928 non-null object
Amazon ES ID       21101 non-null float64
Amazon ES Path     18800 non-null object
Amazon US ID       21584 non-null float64
Amazon US Path     21550 non-null object
Amazon CA ID       16679 non-null float64
Amazon CAN Path    10409 non-null object
Amazon AU ID       21234 non-null float64
Amazon AU Path     21213 non-null object
Ebay UK ID         21103 non-null float64
Ebay UK Path       21087 non-null object
Ebay DE ID         18343 non-null float64
Ebay DE Path       13

In [7]:
master_path=data_frame['Master Path']
master_path[0]

'/Categories/3D Printing & Scanning/3D Printer Parts & Accessories/3D Printer Accessories'

In [8]:
path_list=master_path[0].split('/')
path_list

['',
 'Categories',
 '3D Printing & Scanning',
 '3D Printer Parts & Accessories',
 '3D Printer Accessories']

In [9]:
path_list[-1]

'3D Printer Accessories'

In [10]:
# Save split name keyword from the outermost level of 'Master Path'
var = []
for category in data_frame['Master Path'].str.split('/')[:] :  
    var.append(category[-1])  
data_frame['Master_list'] = var
var

['3D Printer Accessories',
 '3D Printer Controllers',
 '3D Printer Driver Modules',
 '3D Printer Extruder Parts',
 '3D Printer Extruders',
 '3D Printer Interface Modules',
 '3D Printer Motors',
 '3D Printer Platforms',
 '3D Printer Timing Belts',
 '3D Printers',
 'Filament 3D Printing Materials',
 'Granular 3D Printing Materials',
 'Liquid 3D Printing Materials',
 '3D Scanners',
 'Dressing Tools',
 'Mandrels',
 'Wheel Adapters',
 'Wheel Flanges',
 'Abrasive Belts',
 'Bevel Brushes',
 'Cup Brushes',
 'Disc Brushes',
 'End Brushes',
 'Power Brush Sets',
 'Spiral Brushes',
 'Strip Brushes',
 'Tube Brushes',
 'Wheel Brushes',
 'Scratch Brushes',
 'Bands',
 'Cartridge Rolls',
 'Grinding Cones',
 'Grinding Mounted Points',
 'Grinding Plugs',
 'Non-woven Bits',
 'Angle Grinder Wheels',
 'Bench Grinding Wheels',
 'Bristle Discs',
 'Convolute Wheels',
 'Cup Wheels',
 'Cut-off Wheels',
 'Die Grinder Wheels',
 'Fladder Blades',
 'Flap Discs',
 'Flap Wheels',
 'Grinding Discs',
 'Grinding Points',

In [11]:
data_frame[9102:9105]

,ID,Master Path,Amazon UK ID,Amazon UK Path,Amazon DE ID,Amazon DE Path,Amazon FR ID,Amazon FR Path,Amazon IT ID,Amazon IT Path,...,Ebay US,Ebay US Path,Ebay CA ID,Ebay CA Path,Ebay AU ID,Ebay AU Path,Test Channel,Test Path,Unnamed: 36,Master_list
9102,9103,/Categories/Nappy Changing/Nappies/Disposable ...,9.641487e+09,/Categories/Nappy Changing/Nappies/Disposable ...,9.645592e+09,Baby/Wickeln/Windeln/Einwegwindeln/Babywindeln,9.644690e+09,Bébé et Puériculture/Change de Bébé/Couches/Co...,9.645297e+09,Prima infanzia/Cambio del pannolino/Pannolini/...,...,26268.0,NaN,20399.0,Baby > Diapering > Other Diapering,20399,Baby > Nappies > Other Nappies,NaN,Baby > Nappies > Other Nappies,NaN,Baby Nappies
9103,9104,/Categories/Nappy Changing/Nappies/Disposable ...,1.890286e+09,/Categories/Nappy Changing/Nappies/Disposable ...,3.969105e+09,Baby/Wickeln/Windeln/Einwegwindeln/Schwimmwindeln,3.966227e+09,Bébé et Puériculture/Change de Bébé/Couches/Co...,1.806559e+09,Prima infanzia/Cambio del pannolino/Pannolini/...,...,26268.0,NaN,100222.0,Baby > Diapering > Swim Diapers,20399,Baby > Nappies > Other Nappies,NaN,Baby > Nappies > Other Nappies,NaN,Disposable Swim Nappies
9104,9105,/Categories/Nappy Changing/Nappies/Disposable ...,1.890264e+09,/Categories/Nappy Changing/Nappies/Disposable ...,3.969107e+09,Baby/Töpfchen & Trittschemel/Über- & Spreizhosen,3.966228e+09,Bébé et Puériculture/Apprentissage de la propr...,1.806560e+09,Prima infanzia/Cambio del pannolino/Pannolini/...,...,37631.0,Baby > Potty Training,37631.0,Baby > Potty Training,15559,Baby > Nappies > Nappies (Disposable),NaN,Baby > Nappies > Nappies (Disposable),NaN,Disposable Training Pants


In [36]:
# Input (using a mini user interface) exemplary Products by keyword names to match to Master Mapping later on

product_name = input('Type product name (example: Training Pants)')
#product_name = 'Terminal Cables' #  Terminal Cables  blue Disposable Training Pants,
product_name = str(product_name)
product_name_list = product_name.split(' ')
product_name_list

Type product name (example: Training Pants)Training Nappies


['Training', 'Nappies']

In [37]:
# Script to match either word in a new product name string to select best fitting row in the Master table. 
# If there is more good fits then with another mini GUI one can select a correct row/ID from suggested 
# by the algorithm rows
bla_M = [True] * len(data_frame['Master_list'])
result_M = np.array(bla_M)

for word in product_name_list:
    print(word)
    matched_M = data_frame['Master_list'].str.contains(r'\b{}\b'.format(word)) # it needed r'
    matched_M = np.array(matched_M)
    y=any(matched_M.tolist())
    if  y:
        result_M = matched_M & result_M
    y=any(result_M.tolist())   
result_M = result_M.tolist()
hits_Master = [i for i, x in enumerate(result_M) if x]

if      len(hits_Master) == 0: 
        print('+++++++++ There is no record yet for this product name in the system ++++++++++')
        #Room here to elaborate new products inputs to Master Mapper
        hits_Master = []
elif    len(hits_Master) >1:
        print('\nThese are possible Master Mapper matches:\n')
        for i in hits_Master:
            print(i+1)
            print(var[i])
            print(data_frame['Master Path'][i])
            print()
        prompt_feedback = input('Input in the box the right ID number()')
        hits_M = int(prompt_feedback)
        hits_M  -= 1
    
else:    
        hits_M = int(str(hits_Master[0]))
        hits_M += 1 
print('Master_ID(s) = '+str(hits_M))
print('%%%%%%%%%%%% below are matches for channels & regions %%%%%%%%%%%')
print()
dictionary = dict.fromkeys(data_frame.columns) 
dictionary['ID'] = data_frame['ID'][hits_Master[0]]
dictionary['Master Path'] = data_frame['Master Path'][hits_Master[0]]
t = 0
def function(data_frame, k):
    bool_series = pd.isnull(data_frame[k])  
    data_frame[bool_series] = 'emptycell'
    var2 = []
    for category in data_frame[k].str.split('/')[:] : 
        var2.append(category[-1]) 
    return var2
counter = 2
for k in data_frame.columns[3:34:2]: # [3:34:2], tested was [3:4]
    var2 = function(data_frame, k)
    data_frame[k + ' list'] = var2
# About below assignments: I substitue other than UK region data with the UK data since algorithm due to the language mismatch 
# won't find any table data matches. In the SIZE MASTER MAPPER Python code it is showcased how to go around using a
# translation automatically
    if   k == 'Amazon ES Path': 
         k = 'Amazon UK Path'
         t = 1
    elif k ==  'Ebay DE Path':
         k = 'Ebay UK Path'
         t = 2 
    elif k ==  'Amazon CAN Path':
         k =  'Amazon UK Path'
         t = 3   
    elif k ==  'Ebay FR Path': 
         k =  'Ebay UK Path'
         t = 4     
    elif k ==  'Ebay IT Path': 
         k =  'Ebay UK Path'
         t = 5  
    elif k ==  'Amazon IT Path':
         k =  'Amazon UK Path'
         t = 6   
    elif k ==  'Amazon FR Path':
         k =  'Amazon UK Path'
         t = 7  
    bla = [False] * len(data_frame[k + ' list'] )
    result = np.array(bla)

    for word in product_name_list:
        print(word)
        matched = data_frame[k + ' list'] .str.contains(r'\b{}\b'.format(word)) # it needed r'
        matched = np.array(matched)
        y=any(matched.tolist())
        if  y:
            result = matched & result
        y=any(result)
        print(y)
# Some extra tests not to miss a possible match if there is no one in the first try
        if y == False:
            matched = data_frame[k].str.contains('{}'.format(word)) 
            matched = np.array(matched)
            result = matched & result
            y=any(matched.tolist()) 
            print(y)      
    hits = ([i for i, x in enumerate(result) if x])
    print(k)
    print([i + 1 for i, x in enumerate(result) if x])
    print() 
    
    if hits == []:
        for word in product_name_list:
            matched = data_frame[k + ' list'] .str.contains(r'\b{}\b'.format(word)) # it needed r'
       
            matched = np.array(matched)
            y=any(matched.tolist())

            if  y:
                result = matched | result
            y=any(result.tolist())
      
        result = result.tolist()

    
    global_hits = [i for i, x in enumerate(result) if x]
    for i in global_hits:
            print(i+1)
            print(var[i])
            print(data_frame[k + ' list'][i])                         
    hits = min(global_hits, key=lambda x:abs(x - (hits_M - 1)))
    print(hits)   
    if   t == 1:
         k = 'Amazon ES Path'
    elif t == 2:
         k = 'Ebay DE Path'
    elif t == 3:
         k = 'Amazon CAN Path'
    elif t == 4:
         k = 'Ebay FR Path'        
    elif t == 5:
         k = 'Ebay IT Path' 
    elif t == 6:
         k = 'Amazon IT Path'        
    elif t == 7:
         k = 'Amazon FR Path' 
# Save the matching to querred Product Name data rows to a new dataframe df_output
    dictionary[k] = data_frame[k][hits]
    dictionary[data_frame.columns[counter]] =  data_frame[data_frame.columns[counter]][hits]
    counter += 2
    t = 0
df_output = df_output.append(dictionary, ignore_index=True)
df_output['Product Name'] [count] = product_name
count += 1

Training
Nappies

These are possible Master Mapper matches:

905
Training Jerseys
/Categories/Baseball/Clothing/Boys/Training Jerseys

912
Training Jerseys
/Categories/Baseball/Clothing/Girls/Training Jerseys

926
Training Jerseys
/Categories/Baseball/Clothing/Women/Training Jerseys

966
Training Bats
/Categories/Baseball/Training Equipment/Training Bats

1012
Training Equipment
/Categories/Basketball/Training Equipment

2525
Training Wheels
/Categories/Cycling/Accessories/Training Wheels

6061
Training Heads
/Categories/Hair Care/Hairdresser Furnishings/Training Heads



KeyboardInterrupt: 

In [38]:
# Show the df_output frame with one entry through the mini GUI
Master_mapper = df_output 
df_output   

,Product Name,ID,Master Path,Amazon UK ID,Amazon UK Path,Amazon DE ID,Amazon DE Path,Amazon FR ID,Amazon FR Path,Amazon IT ID,...,Ebay CA ID,Ebay CA Path,Ebay AU ID,Ebay AU Path,Test Channel,Test Path,Unnamed: 36,Amazon DE Path list,Amazon UK Path list,Master_list
0,D-Terminal Cables,131,/Categories/Accessories/Home Audio & Video Acc...,528600031.0,/Categories/Accessories/Home Audio & Video Acc...,1.382702e+09,Elektronik & Foto/Fernseher & Heimkino/Zubehör...,16251591.0,High-Tech/Consommables et Accessoires/Accessoi...,473318031.0,...,14964.0,"Consumer Electronics > TV, Video & Home Audio ...",14964,Home Entertainment > Television Accessories > ...,None,None,None,None,None,None


In [41]:
# Save df_output table to an excel file (optionally can be saved as a sub_spreadsheet of existing Master Mapping excel)
Master_mapper.to_excel('Master Mapper.xlsx', header = True)